In [2]:
2+2

4

In [1]:
import json
import pandas as pd
import glob

# Get all JSON files inside the folder
files = glob.glob("Spotify Extended Streaming History/*.json")

data = []

for f in files:
    with open(f, "r", encoding="utf-8") as file:
        data.extend(json.load(file))

df = pd.DataFrame(data)

print(df.head())
print("Total records:", len(df))


                     ts                              platform  ms_played  \
0  2020-05-20T17:43:31Z  Android OS 8.1.0 API 27 (LAVA, Z60s)       3866   
1  2020-05-20T17:44:46Z  Android OS 8.1.0 API 27 (LAVA, Z60s)      75060   
2  2020-05-20T17:48:27Z  Android OS 8.1.0 API 27 (LAVA, Z60s)     220293   
3  2020-05-20T17:48:54Z  Android OS 8.1.0 API 27 (LAVA, Z60s)       5768   
4  2020-05-20T17:48:56Z  Android OS 8.1.0 API 27 (LAVA, Z60s)        497   

  conn_country       ip_addr master_metadata_track_name  \
0           IN  27.97.202.90              Wrecking Ball   
1           IN  27.97.202.90              Wrecking Ball   
2           IN  27.97.202.90       Just The Way You Are   
3           IN  27.97.202.90           Treat You Better   
4           IN  27.97.202.90           Sugar & Brownies   

  master_metadata_album_artist_name master_metadata_album_album_name  \
0                       Miley Cyrus         Bangerz (Deluxe Version)   
1                       Miley Cyrus         

In [2]:
print("Total rows in df =", len(df))


Total rows in df = 79274


In [3]:
before = len(df)
df = df.drop_duplicates()
after = len(df)
print("Duplicates removed:", before - after)


Duplicates removed: 16


In [4]:
df["ts"] = pd.to_datetime(df["ts"])
print("Start:", df["ts"].min())
print("End:", df["ts"].max())


Start: 2020-05-20 17:43:31+00:00
End: 2025-11-16 15:15:58+00:00


In [5]:
df.duplicated().sum()


np.int64(0)

In [18]:
df.drop(columns=['incognito_mode', 'ip_addr','conn_country'], inplace=True)

In [8]:
df.drop(columns=['audiobook_title', 'audiobook_uri','audiobook_chapter_uri','audiobook_chapter_title','spotify_episode_uri'], inplace=True)

In [9]:
df.sample()

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,reason_start,reason_end,shuffle,skipped,offline,incognito_mode
78746,2025-11-03 11:38:48+00:00,android,164489,IN,2401:4900:32fc:29a8:2710:2421:5270:991c,Care,Sonder,Into,spotify:track:3tkxfORwPo2zydAf25YFOc,None,None,trackdone,trackdone,True,False,False,False


In [10]:
df['platform'].unique()

array(['Android OS 8.1.0 API 27 (LAVA, Z60s)',
       'Android OS 7.0 API 24 (Xiaomi, Redmi Note 4)',
       'Android OS 9 API 28 (realme, RMX1971)',
       'Android OS 10 API 29 (realme, RMX1971)',
       'Android OS 11 API 30 (realme, RMX1971)', 'android'], dtype=object)

In [11]:
df.rename(columns={
    "master_metadata_track_name": "track_name",
    "master_metadata_album_artist_name": "artist",
    "master_metadata_album_album_name": "album"
}, inplace=True)


In [12]:
df['ts'] = pd.to_datetime(df['ts'], utc=True)
df['ts'] = df['ts'].dt.tz_convert('Asia/Kolkata')
df["date"] = df["ts"].dt.date
df["time"] = df["ts"].dt.time
df["year"] = df["ts"].dt.year
df["month"] = df["ts"].dt.month
df["day"] = df["ts"].dt.day
df["hour"] = df["ts"].dt.hour
df["weekday"] = df["ts"].dt.weekday


In [13]:
df["min_played"] = df["ms_played"] / (1000 * 60)


In [26]:
df.head()

,platform,track_name,artist,album,episode_name,episode_show_name,reason_start,reason_end,shuffle,skipped,offline,date,time,year,month,day,hour,weekday,min_played
0,lava mobile,Wrecking Ball,Miley Cyrus,Bangerz (Deluxe Version),None,None,clickrow,endplay,False,False,False,2020-05-20,23:13:31,2020,5,20,23,Wednesday,0.064433
1,lava mobile,Wrecking Ball,Miley Cyrus,Bangerz (Deluxe Version),None,None,clickrow,endplay,False,False,False,2020-05-20,23:14:46,2020,5,20,23,Wednesday,1.251000
2,lava mobile,Just The Way You Are,Bruno Mars,Red Hot R & B,None,None,clickrow,trackdone,False,False,False,2020-05-20,23:18:27,2020,5,20,23,Wednesday,3.671550
3,lava mobile,Treat You Better,Shawn Mendes,Illuminate,None,None,trackdone,endplay,False,False,False,2020-05-20,23:18:54,2020,5,20,23,Wednesday,0.096133
4,lava mobile,Sugar & Brownies,DHARIA,Sugar & Brownies,None,None,clickrow,endplay,False,False,False,2020-05-20,23:18:56,2020,5,20,23,Wednesday,0.008283


In [15]:
df.drop([('ts'),('ms_played'),('spotify_track_uri')], axis=1, inplace=True)

In [16]:
platform_map = {
    "android": "realme mobile",
    "Android OS 8.1.0 API 27 (LAVA, Z60s)": "lava mobile",
    "Android OS 7.0 API 24 (Xiaomi, Redmi Note 4)": "redmi mobile",
    "Android OS 9 API 28 (realme, RMX1971)": "realme mobile",
    'Android OS 11 API 30 (realme, RMX1971)': "realme mobile",
}

df["platform"] = df["platform"].replace(platform_map)


In [20]:
df['year'].value_counts()

year
2021    15817
2023    13703
2020    13534
2022    12569
2024    12140
2025    11495
Name: count, dtype: int64

In [23]:
df.isna().sum()


platform                 0
track_name               0
artist                   0
album                    0
episode_name         79242
episode_show_name    79242
reason_start             0
reason_end               0
shuffle                  0
skipped                  0
offline                  0
date                     0
time                     0
year                     0
month                    0
day                      0
hour                     0
weekday                  0
min_played               0
dtype: int64

In [22]:
df = df.dropna(subset=['artist', 'track_name','album'])


In [24]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')


C:\Users\sachin\AppData\Local\Temp\ipykernel_11848\1705850665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [25]:

df['weekday'] = df['date'].dt.day_name()


C:\Users\sachin\AppData\Local\Temp\ipykernel_11848\1756247022.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weekday'] = df['date'].dt.day_name()


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79242 entries, 0 to 79273
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   platform           79242 non-null  object        
 1   track_name         79242 non-null  object        
 2   artist             79242 non-null  object        
 3   album              79242 non-null  object        
 4   episode_name       0 non-null      object        
 5   episode_show_name  0 non-null      object        
 6   reason_start       79242 non-null  object        
 7   reason_end         79242 non-null  object        
 8   shuffle            79242 non-null  bool          
 9   skipped            79242 non-null  bool          
 10  offline            79242 non-null  bool          
 11  date               79242 non-null  datetime64[ns]
 12  time               79242 non-null  object        
 13  year               79242 non-null  int32         
 14  month      

In [28]:
df['date'] = df['date'].astype(str)
df[df['date'] == '2025-10-29']


C:\Users\sachin\AppData\Local\Temp\ipykernel_11848\3822872659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].astype(str)


,platform,track_name,artist,album,episode_name,episode_show_name,reason_start,reason_end,shuffle,skipped,offline,date,time,year,month,day,hour,weekday,min_played
78300,realme mobile,Get My Shit Together,Johan Lenox,WDYWTBWYGU,None,None,trackdone,trackdone,True,False,False,2025-10-29,00:01:51,2025,10,29,0,Wednesday,3.168833
78301,realme mobile,paint THE sky,Lil Yachty,Let’s Start Here.,None,None,trackdone,endplay,True,True,False,2025-10-29,00:04:22,2025,10,29,0,Wednesday,2.160067
78302,realme mobile,Come Thru,Drake,Nothing Was The Same,None,None,clickrow,endplay,True,True,False,2025-10-29,00:11:47,2025,10,29,0,Wednesday,7.442800
78303,realme mobile,Stars Align (with Drake),Majid Jordan,Wildest Dreams,None,None,trackdone,trackdone,True,False,False,2025-10-29,00:16:09,2025,10,29,0,Wednesday,4.342000
78304,realme mobile,Un-thinkable (I'm Ready) (feat. Drake) - Remix,Alicia Keys,Un-thinkable (I'm Ready) (feat. Drake),None,None,trackdone,trackdone,True,False,False,2025-10-29,00:20:51,2025,10,29,0,Wednesday,4.700450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78417,realme mobile,Die Hard,Kendrick Lamar,Mr. Morale & The Big Steppers,None,None,trackdone,trackdone,True,False,False,2025-10-29,23:23:37,2025,10,29,23,Wednesday,3.983783
78418,realme mobile,The Way You Make Me Feel - 2012 Remaster,Michael Jackson,Bad,None,None,trackdone,trackdone,True,False,False,2025-10-29,23:28:36,2025,10,29,23,Wednesday,4.968667
78419,realme mobile,Gata Only,FloyyMenor,Gata Only,None,None,trackdone,trackdone,True,False,False,2025-10-29,23:33:15,2025,10,29,23,Wednesday,3.700000
78420,realme mobile,Billian Billian,Guri,Billian Billian,None,None,trackerror,fwdbtn,True,True,False,2025-10-29,23:34:40,2025,10,29,23,Wednesday,1.385067
